In [ ]:
import pandas as pd, json, plotly.express as px

with open("../geojson/lombardia_province.geojson") as f:
    province_geo=json.load(f)

In [ ]:
#Pollutants lists
PM10 = ['PM10', 'PM10 (SM2005)']
NO2 = ['Biossido di Azoto']
PM25 = ['Particelle sospese PM2.5']
CO_8h = ['Monossido di Carbonio']
O3 = ['Ozono']
SO2 = ['Biossido di Zolfo']
C6H6 = ['Benzene']

#Range lists (min-max between all years)
range_PM10 = [18, 55]
range_NO2 = [14, 72]
range_PM25 = [10, 58]
range_CO_8h = [0.2, 1.6]
range_O3 = [12, 64]
range_SO2 = [0, 12]
range_C6H6 = [0.5, 4]

#Years available
years = range(2005, 2019)

In [ ]:
def switch_year(argument):
    switcher = {
        2005: '../csv/lombardia/sensori_aria_2005/2005.csv',
        2006: '../csv/lombardia/sensori_aria_2006/2006.csv',
        2007: '../csv/lombardia/sensori_aria_2007/2007.csv',
        2008: '../csv/lombardia/sensori_aria_2008/2008.csv',
        2009: '../csv/lombardia/sensori_aria_2009/2009.csv',
        2010: '../csv/lombardia/sensori_aria_2010/2010.csv',
        2011: '../csv/lombardia/sensori_aria_2011/2011.csv',
        2012: '../csv/lombardia/sensori_aria_2012/2012.csv',
        2013: '../csv/lombardia/sensori_aria_2013/2013.csv',
        2014: '../csv/lombardia/sensori_aria_2014/2014.csv',
        2015: '../csv/lombardia/sensori_aria_2015/2015.csv',
        2016: '../csv/lombardia/sensori_aria_2016/2016.csv',
        2017: '../csv/lombardia/sensori_aria_2017/2017.csv',
        2018: '../csv/lombardia/sensori_aria_2018/2018.csv'
    }
    year = switcher.get(argument, "Invalid input")
    return year

def generate_map(year, pollutant = [], pollutant_range = []):
    stations = pd.read_csv('../csv/lombardia/Stazioni_qualit__dell_aria(lombardia).csv', encoding='utf-8', sep=',')
    y = switch_year(year)
    chosen_year = pd.read_csv(y, encoding='utf-8', sep=',')
    #Remove broken values
    chosen_year.drop(chosen_year.index[(chosen_year["Valore"] < 0)],axis=0,inplace=True)
    #Remove unwanted pollutants
    for index, row in stations.iterrows():
        if row["NomeTipoSensore"] not in pollutant:
            stations.drop(index, inplace=True)
    result = pd.merge(stations,chosen_year,on='IdSensore')
    #Calculate yearly mean by Province
    df = result.groupby(by=['Provincia'])
    df = df["Valore"].mean().reset_index()
    #Generate map
    fig = px.choropleth_mapbox(df, geojson=province_geo,
                               locations='Provincia',
                               featureidkey='properties.prov_acr',
                               color='Valore',
                               color_continuous_scale="YlOrRd",
                               range_color=(pollutant_range[0], pollutant_range[1]),
                               mapbox_style="carto-positron",
                               zoom=7,
                               center = {"lat": 45.67, "lon": 9.7119},
                               opacity=0.5,
                               labels={'Valore':pollutant[0]+': '+str(year)}
                              )
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

In [ ]:
generate_map(2005, PM10, range_PM10)

In [ ]:
generate_map(2005, NO2, range_NO2)

In [ ]:
generate_map(2005, PM25, range_PM25)

In [ ]:
generate_map(2005, CO_8h, range_CO_8h)

In [ ]:
generate_map(2005, O3, range_O3)

In [ ]:
generate_map(2005, SO2, range_SO2)

In [ ]:
generate_map(2005, C6H6, range_C6H6)